In [23]:
import tensorflow as tf
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
import os
import xml.etree.ElementTree as ET
import random
import datetime

In [24]:
DATA_PATH = './data/VOCdevkit/VOC2007/'
DIRECTORY_ANNOTATIONS = 'Annotations/'
DIRECTORY_IMAGES = 'JPEGImages/'

SHUFFLE = True
RANDOM_SEED = datetime.date.today().year + datetime.date.today().month + datetime.date.today().day 

VOC_LABELS = {
    'none': (0, 'Background'),
    'aeroplane': (1, 'Vehicle'),
    'bicycle': (2, 'Vehicle'),
    'bird': (3, 'Animal'),
    'boat': (4, 'Vehicle'),
    'bottle': (5, 'Indoor'),
    'bus': (6, 'Vehicle'),
    'car': (7, 'Vehicle'),
    'cat': (8, 'Animal'),
    'chair': (9, 'Indoor'),
    'cow': (10, 'Animal'),
    'diningtable': (11, 'Indoor'),
    'dog': (12, 'Animal'),
    'horse': (13, 'Animal'),
    'motorbike': (14, 'Vehicle'),
    'person': (15, 'Person'),
    'pottedplant': (16, 'Indoor'),
    'sheep': (17, 'Animal'),
    'sofa': (18, 'Indoor'),
    'train': (19, 'Vehicle'),
    'tvmonitor': (20, 'Indoor'),
}

cur_idx = 0
perm_idx = None
image_idx = [] #이미지 인덱스 
batch_size = 20
batch_idx

In [13]:
def _read_image_annotation(directory, name):
    
    # Read the image file.
    filename = directory + DIRECTORY_IMAGES + name + '.jpg' #파일네임 설정
    image_data = tf.read_file(filename) #리드 


    # Read the XML annotation file.
    filename = os.path.join(directory, DIRECTORY_ANNOTATIONS, name + '.xml') #엑스엠엘
    tree = ET.parse(filename) #파싱준비
    root = tree.getroot() #루트얻어옴 

    # Image shape.
    size = root.find('size')
    shape = [int(size.find('height').text),
             int(size.find('width').text),
             int(size.find('depth').text)]
    
    # Find annotations.
    bboxes = []
    labels = []
    labels_text = []
    difficult = []
    truncated = []
    for obj in root.findall('object'): #object 다 찾아서 넣음 바운딩박스/라벨(번호)/라벨텍스트(이름)/voc에 있는 특징? 어려운거?/좀짤린거
        label = obj.find('name').text
        labels.append(int(VOC_LABELS[label][0]))
        labels_text.append(label.encode('ascii'))

        if obj.find('difficult'):
            difficult.append(int(obj.find('difficult').text))
        else:
            difficult.append(0)
        if obj.find('truncated'):
            truncated.append(int(obj.find('truncated').text))
        else:
            truncated.append(0)

        bbox = obj.find('bndbox')
        bboxes.append((float(bbox.find('ymin').text) / shape[0],
                       float(bbox.find('xmin').text) / shape[1],
                       float(bbox.find('ymax').text) / shape[0],
                       float(bbox.find('xmax').text) / shape[1]
                       ))
    return image_data, shape, bboxes, labels, labels_text, difficult, truncated

def _shuffle_image_idx():
        perm_idx = [image_idx[i] for i in np.random.permutation(np.arange(len(image_idx)))]
        cur_idx = 0 

def make_batch(batch_size=20, shuffle=True):
    if shuffle: #셔플시.
        if cur_idx + batch_size >= len(image_idx): 
            _shuffle_image_idx()
        batch_idx = perm_idx[cur_idx : cur_idx+batch_size]
        cur_idx += batch_size
    else: #노셔플시 
        if cur_idx + batch_size >= len(image_idx):
            batch_idx = image_idx[cur_idx:] + image_idx[cur_idx+batch_size-len(image_idx)]
            cur_idx += batch_size - len(image_idx)
        else:#가장 기본
            batch_idx = image_idx[cur_idx:cur_idx+batch_size]
            cur_idx += batch_size
@####### 내일 여기부터
    
      

In [34]:
img_name = '000041'
_read_image_annotation(DATA_PATH, img_name)

TypeError: 'int' object is not subscriptable

In [37]:
path = os.path.join(DATA_PATH, DIRECTORY_ANNOTATIONS)
filenames = sorted(os.listdir(path))

if SHUFFLE:
    random.seed(RANDOM_SEED)
    random.shuffle(filenames)


'000009'

In [38]:
for i in range(len(filenames)):
    filename = filenames[i] # '000xxxx.xml' 이 있을거고
    img_name = filename[:-4] # 파일이름으로 바꿔줌. 
    _read_image_annotation(DATA_PATH, filenames) #넣어줌.. image_data, shape, bboxes, labels, labels_text, difficult, truncated 이런 데이터가 옴
    #image_data, shape, bboxes, labels, labels_text, difficult, truncated
    
    make_batch#함수를 만들어줘야할듯. 배치사이즈 + 셔플여부를 받아서 돌려주면될듯 
    
    
    

TypeError: Can't convert 'list' object to str implicitly